In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.scripts import training

In [1]:
import torch

t = torch.randint(0, 10, (3, 2, 4))
t1 = t.permute((1, 0, 2))
print(t.shape)
print(t1.shape)

torch.Size([3, 2, 4])
torch.Size([2, 3, 4])


In [8]:
import math
import torch.nn.functional as F

max_seq_len = 3
embed_dim = 4
device = 'cpu'

In [3]:
position = torch.arange(max_seq_len, dtype=torch.float32).unsqueeze(1)
div_term = torch.exp(torch.arange(0, embed_dim, 2,  dtype=torch.float32) * (-math.log(10000.0) / embed_dim))

positional_encoding = torch.zeros(max_seq_len, embed_dim, device=device)
positional_encoding[:, 0::2] = torch.sin(position * div_term)
positional_encoding[:, 1::2] = torch.cos(position * div_term)
positional_encoding

tensor([[ 0.0000,  1.0000,  0.0000,  1.0000],
        [ 0.8415,  0.5403,  0.0100,  0.9999],
        [ 0.9093, -0.4161,  0.0200,  0.9998]])

In [5]:
t1

tensor([[[5, 8, 5, 0],
         [9, 7, 5, 6],
         [0, 6, 7, 8]],

        [[4, 7, 3, 4],
         [8, 0, 7, 1],
         [9, 2, 3, 6]]])

In [10]:
x = t1 + positional_encoding
x

tensor([[[5.0000, 9.0000, 5.0000, 1.0000],
         [9.8415, 7.5403, 5.0100, 6.9999],
         [0.9093, 5.5839, 7.0200, 8.9998]],

        [[4.0000, 8.0000, 3.0000, 5.0000],
         [8.8415, 0.5403, 7.0100, 1.9999],
         [9.9093, 1.5839, 3.0200, 6.9998]]])

In [12]:
y = F.dropout(x, p=0.1)
y

tensor([[[ 5.5556, 10.0000,  5.5556,  1.1111],
         [10.9350,  8.3781,  5.5667,  7.7777],
         [ 1.0103,  0.0000,  7.8000,  9.9998]],

        [[ 4.4444,  8.8889,  0.0000,  5.5556],
         [ 9.8239,  0.6003,  7.7889,  2.2222],
         [ 0.0000,  1.7598,  3.3556,  7.7776]]])

In [13]:
F.layer_norm(y, normalized_shape=x.shape)

tensor([[[ 0.0858,  1.3174,  0.0858, -1.1459],
         [ 1.5766,  0.8680,  0.0888,  0.7016],
         [-1.1739, -1.4539,  0.7078,  1.3174]],

        [[-0.2222,  1.0095, -1.4539,  0.0858],
         [ 1.2686, -1.2875,  0.7047, -0.8380],
         [-1.4539, -0.9662, -0.5239,  0.7015]]])

In [3]:
torch.sum(t, 0)

tensor([[19, 11, 20, 11],
        [ 9, 13, 11, 14]])

In [1]:
n = {
    'x' : 2,
    'y' : 3
}
m = {
    'p' : 1,
    **n
}

print(n)
print(m)

{'x': 2, 'y': 3}
{'p': 1, 'x': 2, 'y': 3}


In [25]:
import pandas as pd

df = pd.read_csv('data/quarter_processed_weighted_2000_9/cov_train.csv')
df.groupby('y').size()

y
0    125311
1    321689
dtype: int64

In [1]:
import pandas as pd
import sys
import h5py
import numpy as np

In [4]:
n = 25

for i in range(0, 193):
  if i%n == 0:
    key = []
    cols = [i for i in range(1024)]
    df = pd.DataFrame(columns=cols)
    csv_path = f'data/ProT5_model_embeddings/{i//n}.csv'
    print(csv_path)

  fpath = f'data/ProT5_model_embeddings/per_protein_embeddings_{i}.h5'

  hf = h5py.File(fpath, 'r')

  keys = list(hf.keys())

  for k in keys:
    arr = np.array(hf[k])
    df2 = pd.DataFrame(arr.reshape(1, -1), columns=cols)
    df = pd.concat([df, df2])
    key.append(k)

  if i%n == n-1 or i == 192:
    k_s = pd.Series(key)
    df.insert(loc=0, column='Accession ID', value=k_s)
    print(df.shape)
    df.to_csv(csv_path)

data/ProT5_model_embeddings/0.csv
(50000, 1025)
data/ProT5_model_embeddings/1.csv
(50000, 1025)
data/ProT5_model_embeddings/2.csv
(50000, 1025)
data/ProT5_model_embeddings/3.csv
(50000, 1025)
data/ProT5_model_embeddings/4.csv
(50000, 1025)
data/ProT5_model_embeddings/5.csv
(50000, 1025)
data/ProT5_model_embeddings/6.csv
(50000, 1025)
data/ProT5_model_embeddings/7.csv
(36000, 1025)


In [11]:
df = pd.read_csv('data/ProT5_model_embeddings/0.csv')

for i in range(1, 8):
    path = f'data/ProT5_model_embeddings/{i}.csv'
    df1 = pd.read_csv(path, index_col=[0])
    df = pd.concat([df, df1])

df.shape

(386000, 1026)

In [17]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

KeyError: "['Unnamed: 0'] not found in axis"

In [19]:
df.to_csv('data/ProT5_model_embeddings/Q2_2022.csv', index=False)

In [18]:
df.columns

Index(['Accession ID', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       ...
       '1014', '1015', '1016', '1017', '1018', '1019', '1020', '1021', '1022',
       '1023'],
      dtype='object', length=1025)

In [20]:
df.head(100)

,Accession ID,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,EPI_ISL_11749529,-0.02092,0.04140,0.000757,-0.03930,-0.009090,0.02786,-0.06620,0.008415,0.04984,...,-0.02983,0.002409,0.04324,-0.04712,-0.012640,0.1606,0.01826,-0.000429,-0.00827,-0.05280
1,EPI_ISL_11749529,-0.01512,0.05533,0.010340,-0.03717,-0.000308,0.03119,-0.06750,0.007187,0.05167,...,-0.03102,0.003906,0.04657,-0.03910,0.000709,0.1543,0.01007,0.001760,-0.01883,-0.04706
2,EPI_ISL_11749529,-0.02045,0.04132,0.000376,-0.03897,-0.008640,0.02788,-0.06635,0.008980,0.05060,...,-0.02965,0.002476,0.04318,-0.04678,-0.012200,0.1602,0.01862,-0.000600,-0.00864,-0.05304
3,EPI_ISL_11749529,-0.01880,0.04860,0.004913,-0.03702,-0.002352,0.02763,-0.06537,0.006250,0.05023,...,-0.02829,-0.000146,0.04825,-0.04153,-0.000274,0.1500,0.01404,0.002258,-0.01341,-0.05103
4,EPI_ISL_11749529,-0.02045,0.04132,0.000373,-0.03894,-0.008620,0.02788,-0.06635,0.008980,0.05060,...,-0.02963,0.002472,0.04318,-0.04675,-0.012184,0.1602,0.01860,-0.000592,-0.00866,-0.05304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,EPI_ISL_11749529,-0.02045,0.04132,0.000373,-0.03894,-0.008620,0.02788,-0.06635,0.008980,0.05060,...,-0.02963,0.002472,0.04318,-0.04675,-0.012184,0.1602,0.01860,-0.000592,-0.00866,-0.05304
96,EPI_ISL_11749529,-0.02045,0.04132,0.000373,-0.03894,-0.008620,0.02788,-0.06635,0.008980,0.05060,...,-0.02963,0.002472,0.04318,-0.04675,-0.012184,0.1602,0.01860,-0.000592,-0.00866,-0.05304
97,EPI_ISL_11749529,-0.02045,0.04132,0.000373,-0.03894,-0.008620,0.02788,-0.06635,0.008980,0.05060,...,-0.02963,0.002472,0.04318,-0.04675,-0.012184,0.1602,0.01860,-0.000592,-0.00866,-0.05304
98,EPI_ISL_11749529,-0.02113,0.04312,0.001872,-0.03920,-0.009940,0.02846,-0.06710,0.008120,0.05038,...,-0.02814,0.001566,0.04340,-0.04596,-0.012650,0.1593,0.01840,-0.001138,-0.00987,-0.05173


In [21]:
df1 = pd.read_csv('data/Q2 data/Q2.csv')
df1.head(100)

,Unnamed: 0,ID,Sequence
0,0,EPI_ISL_11635363,MFVFLVLLPLVSSQCVNLITRTQSYTNSFTRGVYYPDKVFRSSVLH...
1,1,EPI_ISL_11635342,MFVFLVLLPLVSSQCVNLITRTQSYTNSFTRGVYYPDKVFRSSVLH...
2,2,EPI_ISL_11635391,MFVFLVLLPLVSSQCVNLITRTQSYTNSFTRGVYYPDKVFRSSVLH...
3,3,EPI_ISL_11635411,MFVFLVLLPLVSSQCVNLITRTQSYTNSFTRGVYYPDKVFRSSVLH...
4,4,EPI_ISL_11635413,MFVFLVLLPLVSSQCVNLITRTQSYTNSFTRGVYYPDKVFRSSVLH...
...,...,...,...
95,95,EPI_ISL_11635474,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
96,96,EPI_ISL_11635516,MFVFLVLLPLVSSQCVNLITRTQSYTNSFTRGVYYPDKVFRSSVLH...
97,97,EPI_ISL_11635478,MFVFLVLLPLVSSQCVNLITRTQSYTNSFTRGVYYPDKVFRSSVLH...
98,98,EPI_ISL_11635508,MFVFLVLLPLVSSQCVNLITRTQSYTNSFTRGVYYPDKVFRSSVLH...


In [4]:
import torch

param_dict = torch.load('checkpoint/6 (copy)/lstm4/best_model/checkpoint.pth')
param_dict.keys()

dict_keys(['epoch', 'total_epoch', 'state_dict', 'optimizer', 'best_training_loss', 'best_val_loss'])

In [6]:
param_dict['state_dict'].keys()

odict_keys(['sum_weight_1', 'sum_weight_2', 'bn11.weight', 'bn11.bias', 'bn11.running_mean', 'bn11.running_var', 'bn11.num_batches_tracked', 'bn12.weight', 'bn12.bias', 'bn12.running_mean', 'bn12.running_var', 'bn12.num_batches_tracked', 'bn13.weight', 'bn13.bias', 'bn13.running_mean', 'bn13.running_var', 'bn13.num_batches_tracked', 'lstm11.weight_ih_l0', 'lstm11.weight_hh_l0', 'lstm11.bias_ih_l0', 'lstm11.bias_hh_l0', 'lstm11.weight_ih_l1', 'lstm11.weight_hh_l1', 'lstm11.bias_ih_l1', 'lstm11.bias_hh_l1', 'lstm11.weight_ih_l2', 'lstm11.weight_hh_l2', 'lstm11.bias_ih_l2', 'lstm11.bias_hh_l2', 'lstm11.weight_ih_l3', 'lstm11.weight_hh_l3', 'lstm11.bias_ih_l3', 'lstm11.bias_hh_l3', 'lstm12.weight_ih_l0', 'lstm12.weight_hh_l0', 'lstm12.bias_ih_l0', 'lstm12.bias_hh_l0', 'lstm12.weight_ih_l1', 'lstm12.weight_hh_l1', 'lstm12.bias_ih_l1', 'lstm12.bias_hh_l1', 'lstm12.weight_ih_l2', 'lstm12.weight_hh_l2', 'lstm12.bias_ih_l2', 'lstm12.bias_hh_l2', 'lstm12.weight_ih_l3', 'lstm12.weight_hh_l3', 'ls